Install YOLO requirements

In [ ]:
#!pip install -r requirements.txt

In [ ]:
#!pip install termcolor


### Imports and subroutines


In [ ]:
from PIL import Image, ImageSequence
import cv2
import numpy as np
import os
import string
import torch
import glob


Directories

In [ ]:
dir_Acquisition = "\path\to\acquisition\directory"
dir_tiffLayers= "\path\to\tifflayers\directory"
dir_focused= "path\to\focused\layers\directory"
dir_cropped = "\path\to\cropped\slices\directory"

#### Count files

In [ ]:
#Function to count files in a directory
def count_files(folder_path):
    count = 0
    for _, _, files in os.walk(folder_path):
        count += len(files)
    return count

#### Load images from folder

In [ ]:
#Function to load images from a directory
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename))
        if img is not None:
            images.append(img)
    return images

#### Read mulitlayered .tiff and export focused layer

In [ ]:
#Function to Read multi-layered .tiff files and one layer with the highest sharpness score
def Tiff_ToStacks(dir_Acquisition,dir_tiffLayers,dir_focused):

    list_sharpness = []
    copy_list= []
    j = 0

    #Loop images in directorz acquisition
    for image in os.listdir(dir_Acquisition):
      os.chdir(dir_Acquisition)
      tiffopen = Image.open(image)
      j = j+1
      os.chdir(dir_tiffLayers)
      #Loop layers of each image
      for i, page in enumerate(ImageSequence.Iterator(tiffopen)):
        page.mode = 'I'
        page.point(lambda i:i*(1./256)).convert('L').save(str(j)+str(i)+'.jpeg')
        #print(str(j)+str(i))

        #find the focused page
          #Navigate to diretory with .tiffLayers
          #Load layers from Image_i to a collection
        collection  = load_images_from_folder(dir_tiffLayers)
        list_sharpness = []
        copy_list= []
        #Loop the collection
      for w in collection:
        gray = cv2.cvtColor(w, cv2.COLOR_BGR2GRAY)
        array = np.asarray(gray)
        gy, gx = np.gradient(array)
        gnorm = np.sqrt(gx**2 + gy**2)
        sharpness = np.average(gnorm)
        list_sharpness.append(sharpness)

        #copy the list of sharpness evaluation to sort and find the sharpest layer
      copy_list = list_sharpness.copy()
      copy_list.sort()
      #print(copy_list)
      #print(copy_list[list_len-1])

      #sharpest image is the layer with highest sharpness score which is the last element of the sorted array
      sharp= copy_list[len(copy_list)-1]
      s= list_sharpness.index(sharp)
      #print(copy_list)
      #print(list_sharpness)
      #print(s)
      #after getting the index of sharpest layer, save it from collection to focused images directory
      img_to_save = collection[s]
      img_path_name = dir_focused+'/'+str(i)+str(j)+'.jpeg'
      cv2.imwrite(img_path_name,img_to_save)
      print(img_path_name)
      #Delete all files in .tiff layers directory to prepare for loading the next image
      for file in os.listdir(dir_tiffLayers):
        if file.endswith('.jpeg'):
          os.remove(file)

#### Sliding window

In [ ]:
#Function to crop image into slices with Neubauer chamber grid size
# using a sliding window looping over the image
def Sliding_Window(image, stepSize, windowSize, e_num):
  #gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
  #Neubauer Grid size in pixels as calculated from image resolution
  h = 1440
  w = 1440
  #start with a small buffer
  #Step size is input from function call
  for j in range(100, image.shape[0]-h, stepSize):
    for i in range(100, image.shape[1]-w, stepSize):
      gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
      roi = gray[j:j+ windowSize[1], i:i+ windowSize[0]]
      cv2.imwrite(str(i)+str(j)+str(e_num)+".png", roi)

#### Count cells from bounding boxes data

#### Find last created folder in \detect directory

In [ ]:
#Find the last created folder in detection output
#In YOLO folders the detection output is often a newly created folder in the directory where detection runs are saved
#The main objective of this function is to find the latest folder in detection runs output directory

def Latest_folder():
    #Go to \detect directory
    %cd \path\to\detection\output

    #get all subdirectories in detect folder of YOLO
    all_subdirs = [d for d in os.listdir('.') if os.path.isdir(d)]
    #print(all_subdirs)

    #find the last created folder
    latest_subdir = max(all_subdirs, key=os.path.getmtime)
    #print(latest_subdir)

    #change current directory to the last created folder
    os.chdir(latest_subdir)

    return latest_subdir

In [ ]:
 #List and return the names of all .txt files within a specified subdirectory
def folder_filenames(target):
    #try to change directory again to last exp (in case it was not successfuly updated in the previous function)
    try:
        os.chdir(target)
    except:
        pass

    inputlst = [os.listdir('labels')]
    filenamelist = []

    for filename in os.listdir('labels'):
        if filename.endswith(".txt"):
            filenamelist.append(filename)
    return filenamelist

In [ ]:
# Function to read all text files in the current directory, to count the number of lines starting with '0' (live) and '1' (dead) in each file,
# and returns two lists: one with the live counts and another with the dead counts for each file.
def Read_txtboundingBoxes():
    live_count=[]
    dead_count=[]
    #Loop over text files in folder
    for filename in glob.glob(os.path.join(os.getcwd(),'*.txt')):
        #open file and read lines
        with open(filename,'r') as f:
            live = 0
            dead = 0
            #read contents of each file
            text =f.readlines()
            #read line by line
            for line in text:
                #first character in each line
                this_class=line[0]
                if this_class.isnumeric():
                    if int(this_class) == 0:
                        live= live+1
                    elif int(this_class)==1:
                        dead = dead+1
            live_count.append(live)
            dead_count.append(dead)
    return live_count,dead_count

#### List average

In [ ]:
#Calculate list average
def list_avg(L):
    sum = 0
    for i in L:
        sum = sum+i
    avg = int(sum / len(L))
    return avg

SyntaxError: leading zeros in decimal integer literals are not permitted; use an 0o prefix for octal integers (<ipython-input-1-b526b6e8fb02>, line 2)

#### Cell Count averages

In [ ]:
#Calculate list average per one corner(Neubauer chamber)
def list_avg_per_corner(L):

    acq_count = count_files(dir_Acquisition)
    crops_count = count_files(dir_cropped)
    #no of crops per corner
    n = int(crops_count/acq_count)

    averages = []

    #L is the list of calculated number of cells
    for i in range(1, len(L), n):
        corner = L[i:i + n]
        average = round(sum(corner) / len(corner))
        averages.append(average)

    return averages

In [ ]:
print(count_files(dir_Acquisition))
print(count_files(dir_cropped))
acq_count = count_files(dir_Acquisition)
crops_count = count_files(dir_cropped)
#no of crops per corner
n = crops_count/acq_count
print(n)

2
0
0.0


## Preprocessing

#### Find focused stack

In [ ]:
#Function Call
Tiff_ToStacks(dir_Acquisition,dir_tiffLayers,dir_focused)

C:\CellCount\YOLO\OfflineYOLO\Focused/101.jpeg
C:\CellCount\YOLO\OfflineYOLO\Focused/102.jpeg


#### Sliding window crop

In [ ]:
#Load focused images into a collection
collection_tocrop  = load_images_from_folder(dir_focused)

#Crop Slices with sliding window
#Navigate to folder where cropped images will be saved
os.chdir(dir_cropped)
for no, img in enumerate(collection_tocrop):
  #gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
  Sliding_Window(img, 400, [1440,1440],no)


 ## Detection


In [ ]:
#Inference from saved best weights, running on test images folder
!python detect.py --agnostic --save-txt  --weights path\to\bestWeights.pt --img 1280 --source path\to\cropped\imageSlices

Error processing line 1 of C:\Users\opto\AppData\Roaming\Python\Python39\site-packages\vision-1.0.0-py3.9-nspkg.pth:

  Traceback (most recent call last):
    File "C:\ProgramData\Anaconda3\lib\site.py", line 169, in addpackage
      exec(line)
    File "<string>", line 1, in <module>
    File "<frozen importlib._bootstrap>", line 562, in module_from_spec
  AttributeError: 'NoneType' object has no attribute 'loader'

Remainder of file ignored
detect: weights=['C:\\CellCount\\YOLO\\OfflineYOLO\\no21_.pt'], source=C:\CellCount\YOLO\OfflineYOLO\Cropped, data=data\coco128.yaml, imgsz=[1280, 1280], conf_thres=0.25, iou_thres=0.45, max_det=1000, device=, view_img=False, save_txt=True, save_conf=False, save_crop=False, nosave=False, classes=None, agnostic_nms=True, augment=False, visualize=False, update=False, project=runs\detect, name=exp, exist_ok=False, line_thickness=3, hide_labels=False, hide_conf=False, half=False, dnn=False, vid_stride=1
YOLOv5  2022-11-19 Python-3.9.13 torch-1.13.1+cp

## Count cells from labels

In [ ]:
#Navigate to the latest folder in \detect
last_exp = Latest_folder()
print(last_exp)

C:\CellCount\YOLO\OfflineYOLO\yolov5-master\runs\detect
exp36


In [ ]:
#get a list of .txt files in the last exp folder
filenames = folder_filenames(last_exp)

In [ ]:
print(folder_filenames(last_exp))

['1001000.txt', '1001001.txt', '1005000.txt', '1005001.txt', '5001000.txt', '5001001.txt', '5005000.txt', '5005001.txt', '9001000.txt', '9001001.txt', '9005000.txt', '9005001.txt']


In [ ]:
#Change path to labels folder
os.chdir('labels')

In [ ]:
#Extract and print count results from .txt files
#Call function to count class occurrencies from .txt files
live_count, dead_count = Read_txtboundingBoxes()
print('Live cells:')

print("Count",live_count,"\ n")
print("Average from all corners: Sum/#of corners: ",list_avg(live_count),"\n")
averages_per_corner = list_avg_per_corner(live_count)
print("Average per corner (list) :",averages_per_corner,"\t", "Average",list_avg(averages_per_corner),"\n")

print('Dead cells:')
print("Count",dead_count,"\n")
print("Average from all corners: Sum/#of corners: ",list_avg(dead_count),"\n")
dead_averages_per_corner = list_avg_per_corner(dead_count)
print("Average per corner (list) :",dead_averages_per_corner,"\t", "Average",list_avg(dead_averages_per_corner),"\n")

Live cells:
Count [230, 262, 239, 238, 240, 247, 252, 229, 251, 241, 276, 232] 

Average from all corners: Sum/#of corners:  244 

Average per corner (list) : [246, 246] 	 Average 246 

Dead cells:
Count [4, 7, 6, 6, 4, 4, 5, 7, 3, 3, 5, 3] 

Average from all corners: Sum/#of corners:  4 

Average per corner (list) : [5, 4] 	 Average 4 



Write results to a text file

In [ ]:
with open('Summary.txt', 'w') as f:
    print("Live cells:")
    print("Count",live_count)
    print("Average:",list_avg(live_count))

    print("Dead cells:")
    print("Count", dead_count)
    print("Average:",list_avg(dead_count))
    f.close()

Live cells:
Count [51, 60, 46, 46, 62, 63, 61, 55, 39, 62, 64, 51, 55, 53, 42, 57, 57, 63, 65, 50, 45, 62, 60, 58, 57, 45, 51, 57, 55, 62, 71, 42, 51, 57, 57, 58, 0]
Average: 53
Dead cells:
Count [8, 5, 8, 8, 3, 2, 5, 2, 4, 5, 1, 5, 7, 4, 8, 5, 1, 2, 4, 2, 4, 3, 0, 4, 6, 3, 7, 4, 2, 1, 3, 3, 4, 2, 2, 1, 0]
Average: 3
